In [28]:
import pandas as pd
import folium
import webbrowser

pd.set_option('display.precision', 2)


data_path_feb = "../Data/01_Courses_usagers/2023_02/"
data_path_jun = "../Data/01_Courses_usagers.2023_06/"

render_map_path = "render_map/"

In [16]:
stations = pd.read_csv(data_path_feb + "All_data_courses.csv", sep=";").loc[:, ['from_stat', 'Latitude_start', 'longitude_start']].drop_duplicates().reset_index()
stations = stations.rename(columns={"from_stat": "nom", "Latitude_start": "lat", "longitude_start": "long"})

print(stations)

        index                                        nom    lat  long
0           0                  Jean Moulin - Paul Doumer  48.86  2.46
1           1                    Lucie Aubrac - Franklin  48.86  2.44
2           2       Docteurs Déjérine - Pte de Montreuil  48.85  2.41
3           3                    Jules Vallès - Charonne  48.85  2.38
4           5  Gare de l'Est - Place du 11 Novembre 1918  48.88  2.36
...       ...                                        ...    ...   ...
1466  2249988                   Quai Panhard et Levassor  48.83  2.38
1467  2282809                  Ney - Porte de Montmartre  48.90  2.34
1468  2655446                      Jean Macé - Faidherbe  48.85  2.38
1469  2790915                        Centenaire - Sorins  48.86  2.43
1470  2972450              Stalingrad - Désiré Chevalier  48.86  2.44

[1471 rows x 4 columns]


In [18]:
avg_loc = stations[['lat', 'long']].mean()
map_paris = folium.Map(locaion=avg_loc, zoom_start=15)

for station in stations.itertuples():
    marker = folium.Marker(location=(station.lat, station.long), tooltip=station.nom)
    marker.add_to(map_paris)

In [27]:
map_paris.save(render_map_path + "map_paris_1.html")
webbrowser.open(render_map_path + "map_paris_1.html")

TypeError: Object of type Series is not JSON serializable